#  1. Data Collection:

In [1]:
# Import Dependancies

import requests
import pandas as pd
from bs4 import BeautifulSoup
import re  

In [2]:
#Read data from ready-to-use csv files, into dataframes

duolingo_df = pd.read_csv("Resources/duolingo_language_report_by_country.csv")
unpopulation_df = pd.read_csv("Resources/unpopulation.csv")
all_endangered_lang_df = pd.read_csv("Resources/clean_data.csv")
widely_spoken_lan_df = pd.read_csv("Resources/languages_by_total_speakers.csv")

In [3]:
#Check the dataframes

print(f"{duolingo_df.head()}")
print(f"{unpopulation_df.head()}")
print(f"{all_endangered_lang_df.head()}")
print(f"{widely_spoken_lan_df.head()}")


       country pop1_2020 pop2_2020 pop1_2021 pop2_2021 pop1_2022 pop2_2022  \
0  Afghanistan   English   Spanish   English   Turkish    German   English   
1      Albania    German   English    German   English    German   English   
2      Algeria   English    French   English    French   English    French   
3      Andorra   English    French   English    French   English    French   
4       Angola   English    French   English    French   English    French   

  pop1_2023 pop2_2023 pop1_2024 pop2_2024  
0   English    German   English    German  
1    German   English   English    German  
2   English    French   English    French  
3   English    French   English   Spanish  
4   English    French   English    French  
   IndicatorId                            IndicatorName  \
0           46  Population by 5-year age groups and sex   
1           46  Population by 5-year age groups and sex   
2           46  Population by 5-year age groups and sex   
3           46  Population by 5

In [4]:
# Use BeautifulSoup for scraping data about Languages_used_on_Internet_websites

# Wikipedia URL
url1 = "https://en.wikipedia.org/wiki/Languages_used_on_the_Internet"

# Fetch the webpage
response = requests.get(url1)
soup = BeautifulSoup(response.text, "html.parser")

# Find the correct table
table = soup.find("table", {"class": "wikitable"})

# Extract table rows
data = []
for row in table.find_all("tr")[1:]:  # Skip header row
    cols = row.find_all("td")
    if len(cols) >= 4:  # Ensure at least 4 columns exist
        language = cols[1].get_text(strip=True)  # Second column: language
        websites_usage = cols[3].get_text(strip=True)  # Fourth column: websites_usage as of Jan 2025

        data.append([language, websites_usage])

# Convert to DataFrame
df_languages_internet = pd.DataFrame(data, columns=["Language", "Websites_usage (%)"])

# Save to CSV
df_languages_internet.to_csv("Resources/content languages for websites (Jan 2025).csv", index=False)

print(f"Scraped {len(df_languages_internet)} languages")

# Display first few rows
df_languages_internet.head()


Scraped 38 languages


,Language,Websites_usage (%)
0,English,49.3%
1,Spanish,6.0%
2,Russian,3.9%
3,German,5.6%
4,French,4.4%


In [5]:
# Use BeautifulSoup for scraping data about official_languages_by_country

# Wikipedia URL
url2 = "https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory"

# Fetch the webpage
response = requests.get(url2)
soup = BeautifulSoup(response.text, "html.parser")

# Find the first table (it contains the country-language data)
table = soup.find("table", {"class": "wikitable"})

# Extract table data
data = []
for row in table.find_all("tr")[1:]:  # Skip the header row
    cols = row.find_all("td")
    if len(cols) >= 2:
        country = cols[0].get_text(strip=True)  # First column: Country
        languages = cols[2].get_text("\n", strip=True)  # Second column: Official Languages (preserve line breaks)

        # Split languages by new lines 
        for lang in languages.split("\n"):
            # Remove numbering (e.g., "1. English" → "English")
            lang_cleaned = re.sub(r"^\d+\.\s*", "", lang).strip()
            if lang_cleaned:  # Only add non-empty values
                data.append([country, lang_cleaned])

# Convert to DataFrame
df_languages = pd.DataFrame(data, columns=["Country", "Official Language"])

# Save to CSV
df_languages.to_csv("Resources/country_per_official_language.csv", index=False)

print(f"Scraped {len(df_languages)} country_per_official_language'")

# Display first few rows
df_languages.head()

Scraped 532 country_per_official_language'


,Country,Official Language
0,Abkhazia[a],Abkhaz
1,Abkhazia[a],Russian
2,Afghanistan[1],Persian (Dari)
3,Afghanistan[1],Pashto
4,Albania[2],Albanian


In [6]:
# Use BeautifulSoup for scraping data about countries_by_population_growth_rate

# URL of the Wikipedia page
url3 = 'https://en.wikipedia.org/wiki/List_of_countries_by_population_growth_rate'

# Send a GET request to the URL
response = requests.get(url3)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table', {'class': 'wikitable'})

    if tables:
        
        table = tables[0]
        
        # Initialize lists to store the country names and percentage rates
        countries = []
        growth_rates = []

        # Iterate over the rows of the table
        for row in table.find_all('tr')[1:]:  # Skip the header row
            cells = row.find_all(['th', 'td'])
            if len(cells) > 1:
                country = cells[0].get_text(strip=True)  # Scrape country names
                growth_rate = cells[1].get_text(strip=True)  # Scrape growth rates
                countries.append(country)
                growth_rates.append(growth_rate)
        
        # Create a DataFrame from the lists
        pop_growth_df = pd.DataFrame({
            'Country': countries,
            'Growth Rate (%)': growth_rates
        })

        # Save the DataFrame to a CSV file
        pop_growth_df.to_csv('Resources/population_growth_rates.csv', index=False)



pop_growth_df = pop_growth_df.iloc[2:].reset_index(drop=True)

# Remove the star symbol from the country names
pop_growth_df['Country'] = pop_growth_df['Country'].str.replace('*', '', regex=False)

# Display the DataFrame
pop_growth_df.head()


,Country,Growth Rate (%)
0,Afghanistan,2.22
1,Albania,0.16
2,Algeria,1.54
3,Andorra,-0.12
4,Angola,3.33


#  2. Data Cleaning & Pre-processing:

### A. Widely spoken languages dataset

In [7]:
# Drop the unnecessary columns

widely_spoken_lan_df = widely_spoken_lan_df.drop(columns=['Family', 'Branch', 'First-language (L1) speakers', 'Second-language (L2) speakers'])

# Display the DataFrame to check the changes

widely_spoken_lan_df.head()

,Language,Total speakers (L1+L2)
0,English(excl.creole languages),1.515 billion
1,"Mandarin Chinese(incl.Standard Chinese, but ex...",1.140 billion
2,Hindi(excl.Urdu),609 million
3,Spanish(excl.creole languages),560 million
4,Modern Standard Arabic(excl.dialects),332 million


In [8]:
# Correct column name

widely_spoken_lan_df.rename(columns={"Total speakers (L1+L2)": "Total_Speakers"}, inplace=True)

#Check df changes

widely_spoken_lan_df.head()

,Language,Total_Speakers
0,English(excl.creole languages),1.515 billion
1,"Mandarin Chinese(incl.Standard Chinese, but ex...",1.140 billion
2,Hindi(excl.Urdu),609 million
3,Spanish(excl.creole languages),560 million
4,Modern Standard Arabic(excl.dialects),332 million


In [9]:
#convert Total_Speakers values  to numbers

def convert_speaker_count(value):
    if isinstance(value, str):
        value = value.lower().replace(" billion", "e9").replace(" million", "e6")
        return float(eval(value))  # Convert string to float
    return value

widely_spoken_lan_df["Total_Speakers"] = widely_spoken_lan_df["Total_Speakers"].apply(convert_speaker_count)

#Check df changes

widely_spoken_lan_df.head()


,Language,Total_Speakers
0,English(excl.creole languages),1.515000e+09
1,"Mandarin Chinese(incl.Standard Chinese, but ex...",1.140000e+09
2,Hindi(excl.Urdu),6.090000e+08
3,Spanish(excl.creole languages),5.600000e+08
4,Modern Standard Arabic(excl.dialects),3.320000e+08


In [10]:
# Fill Missing Values with an estimated number 1000

widely_spoken_lan_df["Total_Speakers"] = widely_spoken_lan_df["Total_Speakers"].fillna(1000)

#Check df changes

widely_spoken_lan_df

,Language,Total_Speakers
0,English(excl.creole languages),1.515000e+09
1,"Mandarin Chinese(incl.Standard Chinese, but ex...",1.140000e+09
2,Hindi(excl.Urdu),6.090000e+08
3,Spanish(excl.creole languages),5.600000e+08
4,Modern Standard Arabic(excl.dialects),3.320000e+08
5,French(excl.creole languages),3.120000e+08
6,Bengali,2.780000e+08
7,Portuguese(excl.creole languages),2.640000e+08
8,Russian,2.550000e+08
9,Urdu(excl.Hindi),2.380000e+08


### B. Endangered languages dataset

In [11]:
# Drop the unnecessary columns
Endangered_df = all_endangered_lang_df.drop(columns=['Unnamed: 0', 'ID', 'Latitude', 'Longitude', 'Description of the location'])

# Display the DataFrame to check the changes
Endangered_df.head()

,Name in English,Countries,Degree of endangerment,Number of speakers
0,South Italian,Italy,Vulnerable,7500000.0
1,Sicilian,Italy,Vulnerable,5000000.0
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",Vulnerable,4800000.0
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",Vulnerable,4000000.0
4,Lombard,"Italy, Switzerland",Definitely endangered,3500000.0


In [12]:
# Save DataFrame to CSV

Endangered_df.to_csv('Resources/Endangered_Languages.csv', index=False)

In [13]:
# Rename the "Languages" column

Endangered_df.rename(columns={"Name in English": "Language"}, inplace=True)

#Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers
0,South Italian,Italy,Vulnerable,7500000.0
1,Sicilian,Italy,Vulnerable,5000000.0
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",Vulnerable,4800000.0
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",Vulnerable,4000000.0
4,Lombard,"Italy, Switzerland",Definitely endangered,3500000.0


In [14]:
# Fill Missing Values with an estimated  number 1000

Endangered_df["Number of speakers"] = Endangered_df["Number of speakers"].fillna(1000)

# Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers
0,South Italian,Italy,Vulnerable,7500000.0
1,Sicilian,Italy,Vulnerable,5000000.0
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",Vulnerable,4800000.0
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",Vulnerable,4000000.0
4,Lombard,"Italy, Switzerland",Definitely endangered,3500000.0


In [15]:
# Update endangerment labels to be consistent (lowercase, no extra spaces)

Endangered_df["Degree of endangerment"] = Endangered_df["Degree of endangerment"].str.strip().str.lower()

# Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers
0,South Italian,Italy,vulnerable,7500000.0
1,Sicilian,Italy,vulnerable,5000000.0
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000.0
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000.0
4,Lombard,"Italy, Switzerland",definitely endangered,3500000.0


In [16]:
#map categories to numerical values

endangerment_mapping = {
    "safe": 0,
    "vulnerable": 1,
    "definitely endangered": 2,
    "severely endangered": 3,
    "critically endangered": 4,
    "extinct": 5
}

Endangered_df["Endangerment Level"] = Endangered_df["Degree of endangerment"].map(endangerment_mapping)

# Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level
0,South Italian,Italy,vulnerable,7500000.0,1
1,Sicilian,Italy,vulnerable,5000000.0,1
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000.0,1
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000.0,1
4,Lombard,"Italy, Switzerland",definitely endangered,3500000.0,2


In [17]:
# Clean Languages names by removing extra text inside parentheses

def clean_language_name(lang):
    return re.sub(r"\s*\(.*?\)", "", lang).strip()

Endangered_df["Language"] = Endangered_df["Language"].apply(clean_language_name)

# Check df changes

Endangered_df

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level
0,South Italian,Italy,vulnerable,7500000.0,1
1,Sicilian,Italy,vulnerable,5000000.0,1
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000.0,1
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000.0,1
4,Lombard,"Italy, Switzerland",definitely endangered,3500000.0,2
...,...,...,...,...,...
2717,Yonaguni,Japan,severely endangered,1000.0,3
2718,Yucuna,Colombia,definitely endangered,1000.0,2
2719,Yurutí,Colombia,definitely endangered,1000.0,2
2720,Zangskari,"India, Pakistan",definitely endangered,1000.0,2


In [18]:
# Convert to numeric, forcing errors to NaN

Endangered_df["Number of speakers"] = pd.to_numeric(Endangered_df["Number of speakers"], errors="coerce")

# Convert from float to int (after filling NaNs)

Endangered_df["Number of speakers"] = Endangered_df["Number of speakers"].fillna(0).astype(int)

# Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level
0,South Italian,Italy,vulnerable,7500000,1
1,Sicilian,Italy,vulnerable,5000000,1
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000,1
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000,1
4,Lombard,"Italy, Switzerland",definitely endangered,3500000,2


In [19]:
# Convert to numeric, forcing errors to NaN

Endangered_df["Endangerment Level"] = pd.to_numeric(Endangered_df["Endangerment Level"], errors="coerce")

# Convert from float to int (after filling NaNs)

Endangered_df["Endangerment Level"] = Endangered_df["Endangerment Level"].fillna(0).astype(int)

# Check df changes

Endangered_df.head()

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level
0,South Italian,Italy,vulnerable,7500000,1
1,Sicilian,Italy,vulnerable,5000000,1
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000,1
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000,1
4,Lombard,"Italy, Switzerland",definitely endangered,3500000,2


In [20]:
# Check df characteristics

Endangered_df.info()
Endangered_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2722 entries, 0 to 2721
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Language                2722 non-null   object
 1   Countries               2721 non-null   object
 2   Degree of endangerment  2722 non-null   object
 3   Number of speakers      2722 non-null   int32 
 4   Endangerment Level      2722 non-null   int32 
dtypes: int32(2), object(3)
memory usage: 85.2+ KB


,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level
0,South Italian,Italy,vulnerable,7500000,1
1,Sicilian,Italy,vulnerable,5000000,1
2,Low Saxon,"Germany, Denmark, Netherlands, Poland, Russian...",vulnerable,4800000,1
3,Belarusian,"Belarus, Latvia, Lithuania, Poland, Russian Fe...",vulnerable,4000000,1
4,Lombard,"Italy, Switzerland",definitely endangered,3500000,2


In [21]:
# Save the cleaned df in a new csv file

Endangered_df.to_csv('Resources/Speakers_per_Language.csv', index=False)

In [22]:
# Merge "widely_spoken_lan_df" and "Endangered_df", and store the merged dataset in a new df "df_languages"

df_languages = pd.merge(Endangered_df, widely_spoken_lan_df, on="Language", how="outer", suffixes=("_endangered", "_widely_spoken"))

# Check the new df 

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers
0,!Gan!ne,South Africa,extinct,0.0,5.0,NaN
1,A'ingae / Cofan,Ecuador,definitely endangered,700.0,2.0,NaN
2,A'ingae / Cofan,Colombia,severely endangered,379.0,3.0,NaN
3,A'tong,India,severely endangered,1000.0,3.0,NaN
4,Aasax,United Republic of Tanzania,extinct,0.0,5.0,NaN
...,...,...,...,...,...,...
2754,ǁGana,Botswana,vulnerable,1550.0,1.0,NaN
2755,ǁKu ǁ'e,South Africa,extinct,0.0,5.0,NaN
2756,ǁKx'au,South Africa,extinct,0.0,5.0,NaN
2757,ǁXegwi,South Africa,extinct,0.0,5.0,NaN


In [23]:
# Check if both columns are identical
df_languages["Duplicate_Columns"] = df_languages["Number of speakers"] == df_languages["Total_Speakers"]

# Count how many values are identical
df_languages["Duplicate_Columns"].value_counts()

Duplicate_Columns
False    2759
Name: count, dtype: int64

In [24]:
# Clear special characters from language column

def clean_language_name(name):
    return re.sub(r"[^\w\s]", "", str(name)).strip()

df_languages["Language"] = df_languages["Language"].apply(clean_language_name)

# Check df changes

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,Ganne,South Africa,extinct,0.0,5.0,NaN,False
1,Aingae Cofan,Ecuador,definitely endangered,700.0,2.0,NaN,False
2,Aingae Cofan,Colombia,severely endangered,379.0,3.0,NaN,False
3,Atong,India,severely endangered,1000.0,3.0,NaN,False
4,Aasax,United Republic of Tanzania,extinct,0.0,5.0,NaN,False
...,...,...,...,...,...,...,...
2754,ǁGana,Botswana,vulnerable,1550.0,1.0,NaN,False
2755,ǁKu ǁe,South Africa,extinct,0.0,5.0,NaN,False
2756,ǁKxau,South Africa,extinct,0.0,5.0,NaN,False
2757,ǁXegwi,South Africa,extinct,0.0,5.0,NaN,False


In [25]:
# Fill Missing Values with an estimated  number 1000, and copy clean values to Total_Speakers column

df_languages["Number of speakers"].fillna(1000)

df_languages["Total_Speakers"] = df_languages["Total_Speakers"].fillna(df_languages["Number of speakers"])

# Check df changes

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,Ganne,South Africa,extinct,0.0,5.0,0.0,False
1,Aingae Cofan,Ecuador,definitely endangered,700.0,2.0,700.0,False
2,Aingae Cofan,Colombia,severely endangered,379.0,3.0,379.0,False
3,Atong,India,severely endangered,1000.0,3.0,1000.0,False
4,Aasax,United Republic of Tanzania,extinct,0.0,5.0,0.0,False
...,...,...,...,...,...,...,...
2754,ǁGana,Botswana,vulnerable,1550.0,1.0,1550.0,False
2755,ǁKu ǁe,South Africa,extinct,0.0,5.0,0.0,False
2756,ǁKxau,South Africa,extinct,0.0,5.0,0.0,False
2757,ǁXegwi,South Africa,extinct,0.0,5.0,0.0,False


In [26]:
# Again and after merging the 2df, map categories to numerical values

endangerment_mapping = {
    "safe": 0,
    "vulnerable": 1,
    "definitely endangered": 2,
    "severely endangered": 3,
    "critically endangered": 4,
    "extinct": 5
}

df_languages["Endangerment Level"] = df_languages["Degree of endangerment"].map(endangerment_mapping)

# Check df changes

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,Ganne,South Africa,extinct,0.0,5.0,0.0,False
1,Aingae Cofan,Ecuador,definitely endangered,700.0,2.0,700.0,False
2,Aingae Cofan,Colombia,severely endangered,379.0,3.0,379.0,False
3,Atong,India,severely endangered,1000.0,3.0,1000.0,False
4,Aasax,United Republic of Tanzania,extinct,0.0,5.0,0.0,False
...,...,...,...,...,...,...,...
2754,ǁGana,Botswana,vulnerable,1550.0,1.0,1550.0,False
2755,ǁKu ǁe,South Africa,extinct,0.0,5.0,0.0,False
2756,ǁKxau,South Africa,extinct,0.0,5.0,0.0,False
2757,ǁXegwi,South Africa,extinct,0.0,5.0,0.0,False


In [27]:
# Clear again the special character after merging df

def clean_language_name(lang):
    return re.sub(r"[^a-zA-Z\s]", "", str(lang)).strip().lower()

df_languages["Language"] = df_languages["Language"].apply(clean_language_name)

# Check df changes

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,ganne,South Africa,extinct,0.0,5.0,0.0,False
1,aingae cofan,Ecuador,definitely endangered,700.0,2.0,700.0,False
2,aingae cofan,Colombia,severely endangered,379.0,3.0,379.0,False
3,atong,India,severely endangered,1000.0,3.0,1000.0,False
4,aasax,United Republic of Tanzania,extinct,0.0,5.0,0.0,False
...,...,...,...,...,...,...,...
2754,gana,Botswana,vulnerable,1550.0,1.0,1550.0,False
2755,ku e,South Africa,extinct,0.0,5.0,0.0,False
2756,kxau,South Africa,extinct,0.0,5.0,0.0,False
2757,xegwi,South Africa,extinct,0.0,5.0,0.0,False


In [28]:
# Convert to numeric, forcing errors to NaN
df_languages["Total_Speakers"] = pd.to_numeric(df_languages["Total_Speakers"], errors="coerce")

# Convert from float to int (after filling NaNs)
df_languages["Total_Speakers"] = df_languages["Total_Speakers"].fillna(0).astype(int)

# Convert to numeric, forcing errors to NaN
df_languages["Number of speakers"] = pd.to_numeric(df_languages["Number of speakers"], errors="coerce")

# Convert from float to int (after filling NaNs)
df_languages["Number of speakers"] = df_languages["Number of speakers"].fillna(0).astype(int)

# Convert to numeric, forcing errors to NaN
df_languages["Endangerment Level"] = pd.to_numeric(df_languages["Endangerment Level"], errors="coerce")

# Convert from float to int (after filling NaNs)
df_languages["Endangerment Level"] = df_languages["Endangerment Level"].fillna(0).astype(int)

In [29]:
# Check df changes

df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,ganne,South Africa,extinct,0,5,0,False
1,aingae cofan,Ecuador,definitely endangered,700,2,700,False
2,aingae cofan,Colombia,severely endangered,379,3,379,False
3,atong,India,severely endangered,1000,3,1000,False
4,aasax,United Republic of Tanzania,extinct,0,5,0,False
...,...,...,...,...,...,...,...
2754,gana,Botswana,vulnerable,1550,1,1550,False
2755,ku e,South Africa,extinct,0,5,0,False
2756,kxau,South Africa,extinct,0,5,0,False
2757,xegwi,South Africa,extinct,0,5,0,False


In [30]:
# Check for duplicate languages

duplicates = df_languages[df_languages.duplicated(subset=["Language", "Countries"], keep=False)]
print(duplicates)

       Language                 Countries Degree of endangerment  \
30         ainu        Russian Federation                extinct   
31         ainu        Russian Federation                extinct   
57        aleut  United States of America  critically endangered   
58        aleut  United States of America    severely endangered   
110     arapaho  United States of America    severely endangered   
...         ...                       ...                    ...   
2511        ute  United States of America    severely endangered   
2512        ute  United States of America    severely endangered   
2513        ute  United States of America    severely endangered   
2617  winnebago  United States of America    severely endangered   
2618  winnebago  United States of America    severely endangered   

      Number of speakers  Endangerment Level  Total_Speakers  \
30                     0                   5               0   
31                     0                   5           

In [31]:
# Drop duplicate rows 

df_languages = df_languages.drop_duplicates(subset=["Language", "Countries"], keep="first")


In [32]:
# Check for duplicate one more time

duplicates = df_languages[df_languages.duplicated(subset=["Language", "Countries"], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [Language, Countries, Degree of endangerment, Number of speakers, Endangerment Level, Total_Speakers, Duplicate_Columns]
Index: []


In [33]:
# Check if both columns are identical
df_languages["Duplicate_Columns"] = df_languages["Number of speakers"] == df_languages["Total_Speakers"]

# Count how many values are identical
df_languages["Duplicate_Columns"].value_counts()

C:\Users\azerty\AppData\Local\Temp\ipykernel_21088\2495572796.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_languages["Duplicate_Columns"] = df_languages["Number of speakers"] == df_languages["Total_Speakers"]


Duplicate_Columns
True     2662
False      37
Name: count, dtype: int64

In [34]:
# Display rows where the values are different

df_languages[df_languages["Duplicate_Columns"] == False]

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
71,amharic,NaN,NaN,0,0,60000000,False
263,bengali,NaN,NaN,0,0,278000000,False
275,bhojpuri,NaN,NaN,0,0,53000000,False
622,egyptian arabicexclother arabic dialects,NaN,NaN,0,0,103000000,False
636,englishexclcreole languages,NaN,NaN,0,0,1515000000,False
668,frenchexclcreole languages,NaN,NaN,0,0,312000000,False
753,gujarati,NaN,NaN,0,0,63000000,False
786,hausa,NaN,NaN,0,0,88000000,False
801,hindiexclurdu,NaN,NaN,0,0,609000000,False
856,indonesianexcl othermalay,NaN,NaN,0,0,199000000,False


In [35]:
# Drop rows where the values are different

df_languages = df_languages[df_languages["Duplicate_Columns"]]
df_languages

,Language,Countries,Degree of endangerment,Number of speakers,Endangerment Level,Total_Speakers,Duplicate_Columns
0,ganne,South Africa,extinct,0,5,0,True
1,aingae cofan,Ecuador,definitely endangered,700,2,700,True
2,aingae cofan,Colombia,severely endangered,379,3,379,True
3,atong,India,severely endangered,1000,3,1000,True
4,aasax,United Republic of Tanzania,extinct,0,5,0,True
...,...,...,...,...,...,...,...
2754,gana,Botswana,vulnerable,1550,1,1550,True
2755,ku e,South Africa,extinct,0,5,0,True
2756,kxau,South Africa,extinct,0,5,0,True
2757,xegwi,South Africa,extinct,0,5,0,True


In [36]:
# Store them in a new df

df_different_values = df_languages[df_languages["Duplicate_Columns"] == False]

df_different_values.to_csv("Resources/different_values.csv", index=False)

In [37]:
# Read the clean dataset of updated different values

df_different_values_modified = pd.read_csv("Resources/Man_modif_widely_spoken_lang.csv")

df_different_values_modified.head()

,Language,Countries,Degree of endangerment,Number of speakers,Total_Speakers,Duplicate_Columns,Endangerment_Level
0,amharic,Ethiopia,Safe,60000000,60000000,False,0
1,arabic,Many,Safe,435000000,435000000,False,0
2,bengali,Bangladesh,Safe,278000000,278000000,False,0
3,bhojpuri,India,Safe,53000000,53000000,False,0
4,chinese,China,Safe,1361000000,1361000000,False,0


In [38]:
# Concatenate corrected and initial dataframes

df_languages_final = pd.concat([df_languages, df_different_values_modified], ignore_index=True)

In [39]:
# Compare content of columns "Number of speakers" and "Total_Speakers"

if df_languages_final['Number of speakers'].equals(df_languages_final['Total_Speakers']):
    print("The columns have exactly the same data.")
else:
    print("The columns do not have the same data.")

The columns have exactly the same data.


In [40]:
# Save the final dataset

df_languages_final.to_csv("Resources/df_language_final.csv", index=False)